In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression

In [4]:
# 1. Membaca file CSV
file_path = 'Dataset_Sentimen_Emosi.csv'
data = pd.read_csv(file_path)

In [6]:
# 2. Membersihkan teks
def clean_text(text):
    text = re.sub(r'\d+', '', text)  # Hapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Hapus tanda baca
    text = text.lower()  # Ubah menjadi huruf kecil
    return text

In [8]:
data = data.dropna(subset=['Tweet', 'Emosi'])  # Hapus baris dengan nilai hilang
data['Tweet'] = data['Tweet'].apply(clean_text)
data['Emosi'] = data['Emosi'].astype(int)  # Pastikan tipe data 'Emosi' adalah integer

In [10]:
# 3. Menyeimbangkan data dengan oversampling
X = data['Tweet']
y = data['Emosi']

In [12]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3), max_features=7000, max_df=0.8)
X_tfidf = vectorizer.fit_transform(X)

In [14]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_tfidf, y)

In [16]:
# 4. Membagi data menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [18]:
# 5. Melatih Logistic Regression
log_reg_model = LogisticRegression(max_iter=200, solver='lbfgs', class_weight='balanced', random_state=42)
log_reg_model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=200, random_state=42)

In [20]:
# 6. Evaluasi performa model Logistic Regression
y_pred_log_reg = log_reg_model.predict(X_test)
print("Accuracy (Logistic Regression):", accuracy_score(y_test, y_pred_log_reg))
print("\nClassification Report (Logistic Regression):\n", classification_report(y_test, y_pred_log_reg))

Accuracy (Logistic Regression): 0.6680497925311203

Classification Report (Logistic Regression):
               precision    recall  f1-score   support

          -2       0.72      0.71      0.72        70
          -1       0.59      0.69      0.64        55
           1       0.65      0.45      0.54        66
           2       0.69      0.86      0.77        50

    accuracy                           0.67       241
   macro avg       0.67      0.68      0.67       241
weighted avg       0.67      0.67      0.66       241

